# Natural Language Processing

## Tugas 1: Regular Expressions & Model Bahasa N-gram

### Mekanisme

Anda hanya diwajibkan untuk mengumpulkan file ini saja ke uploader yang disediakan di https://elearning.uai.ac.id/. Ganti nama file ini saat pengumpulan menjadi **tugas1_NIM.ipynb**.

**Keterlambatan**: Pengumpulan tugas yang melebihi tenggat yang telah ditentukan tidak akan diterima. Keterlambatan akan berakibat pada nilai nol untuk tugas ini.

**Kolaborasi**: Anda diperbolehkan untuk berdiskusi dengan teman Anda, tetapi dilarang keras menyalin kode maupun tulisan dari teman Anda.

### Petunjuk

Pastikan jawaban Anda singkat, padat, dan jelas. Mayoritas pertanyaan yang diberikan dapat dijawab dalam 3-4 kalimat saja.

In [13]:
import numpy as np
import pandas as pd

## 1. Regular Expressions (5 poin)

### Soal 1.1 (2 poin)

Cari pola regular expression yang dapat mencocokkan semua elemen di dalam `words_a`, tetapi tidak cocok dengan apa pun di dalam `words_b`.

In [9]:
import pandas as pd

words_a = [
    "membacakan",
    "menuliskan",
    "melihatkan",
    "mendengarkan",
    "mengajarkan",
    "menarikan",
    "menyanyikan",
    "memasakkan",
    "mencucikan",
    "membelikan",
    "menjualkan",
    "memberikan",
]

words_b = [
    "rumah",
    "meja",
    "kursi",
    "ikan",
    "semen",
    "ramen",
    "akan",
    "bukan",
    "mental",
]

pattern = r"^me(?:ny|ng|n|m|l|r|b)[a-z]+kan$"

assert pd.Series(words_a).str.contains(pattern).all()
assert not pd.Series(words_b).str.match(pattern).any()

### Soal 1.2 (3 poin)

Cari pola regular expression yang dapat mencocokkan semua elemen di dalam `buah_a`, tetapi tidak cocok dengan apa pun di dalam `buah_b`. Anda hanya boleh menggunakan maksimal 6 karakter untuk pola yang dihasilkan.

In [27]:
import pandas as pd

buah_a = [
    "pisang",
    "mangga",
    "rambutan",
    "durian",
    ]

buah_b = [
    "pir",
    "stroberi",
    "ceri",
    "kiwi",
    "leci",
    "persik",
]

pattern = r"an"
assert len(pattern) <= 6
assert pd.Series(buah_a).str.contains(pattern).all()
assert not pd.Series(buah_b).str.contains(pattern).any()

## 2. Model Bahasa N-gram (15 poin)

### Deskripsi Dataset

Dataset yang Anda akan gunakan dalam tugas ini adalah [Indonesian News Corpora 300K](https://wortschatz.uni-leipzig.de/en/download/Indonesian#ind_news_2024) tahun 2024 dari Leipzig Corpora Collection.

In [ ]:
sentences = pd.read_csv(
    "https://raw.githubusercontent.com/aliakbars/uai-nlp/refs/heads/main/datasets/ind_news_2024_300K-sentences.txt",
    sep="\\t",
    names=["index", "text"],
)
del sentences["index"]

### Soal 2.1 (2 poin)

Bagilah dataset menjadi data latih dan data uji dengan rasio 80:20.

In [6]:
sentences = pd.read_csv(
    "https://raw.githubusercontent.com/aliakbars/uai-nlp/refs/heads/main/datasets/ind_news_2024_300K-sentences.txt",
    sep="\\t",
    names=["index", "text"],
    engine="python"
)
del sentences["index"]

from sklearn.model_selection import train_test_split

train, test = train_test_split(sentences, test_size=0.2, random_state=42)

assert len(train) == 240_000
assert len(test) == 60_000

### Soal 2.2 (2 poin)

Buatlah tabel `unigram_df` yang menghasilkan **unigram** dari data latih yang telah dihasilkan di atas. Gunakan [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) untuk proses tokenisasi dan menghasilkan unigramnya, serta pola token default `'(?u)\\b\\w\\w+\\b'`.

Contoh hasil tabel:
| word      |   freq |
|:----------|-------:|
| diketahui |   2654 |
| layanan   |   2015 |
| maju      |   1386 |
| aksi      |   1707 |
| januari   |   1316 |

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
sentences = pd.read_csv(
    "https://raw.githubusercontent.com/aliakbars/uai-nlp/refs/heads/main/datasets/ind_news_2024_300K-sentences.txt",
    sep="\\t",
    names=["index", "text"],
    engine="python"
)
del sentences["index"]

vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w\w+\b')
X_train = vectorizer.fit_transform(train["text"])

unigram_df = pd.DataFrame({
    "word": vectorizer.get_feature_names_out(),
    "freq": X_train.sum(axis=0).A1
}).sort_values(by="freq", ascending=False)

assert ["word", "freq"] == unigram_df.columns.to_list()

### Soal 2.3 (3 poin)

Buatlah tabel `bigram_df` yang menghasilkan **bigram** dari data latih yang telah dihasilkan di atas. Gunakan [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) untuk proses tokenisasi dan menghasilkan bigram, serta pola token default `'(?u)\\b\\w\\w+\\b'`. Tambahkan `ooo` sebagai _start token_ dan `zzz` sebagai _end token_.

Contoh hasil tabel:
| word          | next_word   |   freq |
|:--------------|:------------|-------:|
| akan          | ada         |    461 |
| sebagai       | bentuk      |    365 |
| dua           | orang       |    363 |
| internasional | zzz         |    385 |
| kasus         | ini         |    535 |
| kami          | berharap    |    379 |
| informasi     | yang        |    398 |
| yang          | kita        |    465 |
| pada          | musim       |    325 |
| tahun         | zzz         |   1060 |

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

train_modified = train["text"].apply(lambda x: f"ooo {x.strip()} zzz")

vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w\w+\b')
X_bigram = vectorizer.fit_transform(train_modified)

bigram_terms = vectorizer.get_feature_names_out()
bigram_freq = X_bigram.sum(axis=0).A1

bigram_split = [term.split() for term in bigram_terms]

bigram_df = pd.DataFrame(bigram_split, columns=["word", "next_word"])
bigram_df["freq"] = bigram_freq
bigram_df = bigram_df.sort_values(by="freq", ascending=False).reset_index(drop=True)

assert ["word", "next_word", "freq"] == bigram_df.columns.to_list()
assert "ooo" in bigram_df["word"].to_list()
assert "zzz" in bigram_df["next_word"].to_list()

### Soal 2.4 (5 poin)

Implementasikan kode yang menghasilkan array berisi skor (*log probability*) jika diberikan sebuah kalimat. Petunjuk implementasi:
1. Gunakan tabel unigram dan bigram yang telah dihasilkan di soal sebelumnya.
1. Gunakan metode Stupid backoff yang menggunakan parameter `alpha` untuk mengatasi kasus bigram yang tidak ditemukan.
1. Gunakan metode Laplace smoothing untuk menangani kasus unigram yang tidak ditemukan.

In [13]:
import numpy as np
class BigramModel:
    def __init__(
        self, unigram_df: pd.DataFrame, bigram_df: pd.DataFrame, alpha: float = 0.4
    ):
        self.unigram_df = unigram_df.set_index("word")
        self.bigram_df = bigram_df.set_index(["word", "next_word"])
        self.alpha = alpha
        self.total_unigrams = self.unigram_df["freq"].sum()
        self.vocab_size = len(self.unigram_df)

    def score_unigram(self, word: str) -> float:
        count = self.unigram_df["freq"].get(word, 0)
        prob = (count + 1) / (self.total_unigrams + self.vocab_size)  # Laplace smoothing
        return np.log(prob)

    def score(self, word: str, prev_word: str) -> float:
        bigram = (prev_word, word)
        if bigram in self.bigram_df.index:
            bigram_count = self.bigram_df.loc[bigram, "freq"]
            prev_count = self.unigram_df["freq"].get(prev_word, 0)
            if prev_count == 0:
                return self.score_unigram(word)
            prob = bigram_count / prev_count
            return np.log(prob)
        else:
            return np.log(self.alpha) + self.score_unigram(word)

    def sentence_score(self, sentence: str, n: int = 2) -> np.ndarray:
        tokens = ["ooo"] + sentence.strip().split() + ["zzz"]
        scores = []
        for i in range(1, len(tokens)):
            if n == 2:
                scores.append(self.score(tokens[i], tokens[i - 1]))
            elif n == 1:
                scores.append(self.score_unigram(tokens[i]))
            else:
                raise NotImplementedError("Saat ini hanya bisa untuk n=1 atau n=2")
        return np.array(scores)


bm = BigramModel(unigram_df, bigram_df, alpha=0.4)
assert bm.score_unigram("qqq") > bm.score("qqq", "xxx")
assert bm.score("qqq", "xxx") == np.log(
    0.4 * 1 / (unigram_df.freq.sum() + len(unigram_df))
)
assert bm.score("mana", "di") <= 0
assert bm.score("mungkin", "tidak") > bm.score("qqq", "xxx")
assert bm.score("satu", "salah") > bm.score("tangkap", "salah")
assert (bm.sentence_score("salah satu") <= 0).all()

### Soal 2.5.a (2 poin)

Implementasikan fungsi untuk menghitung perplexity dalam logaritma. Fungsi ini hanya menerima masukan berupa array berisi $\log P(w_i)$.

$$
PP(W) = \sqrt[N]{\frac{1}{\prod_{i=1}^N P(w_i)}}
$$

In [15]:
import numpy as np
def perplexity(log_probs: np.ndarray):
    N = len(log_probs)
    if N == 0:
        return 1
    perplexity_value = np.exp(-np.sum(log_probs) / N)
    return perplexity_value

assert perplexity(np.array([0, 0, 0])) == 1
assert perplexity(np.array([-10, -20])) == np.exp(15)

### Soal 2.5.b (1 poin)

Berdasarkan kalimat contoh yang diberikan, periksa dan pastikan bahwa perplexity dari `BigramModel` lebih kecil daripada model unigram.

In [23]:
import numpy as np

def calculate_perplexity(model, sentence, n=2):
    tokens = ["ooo"] + sentence.strip().split() + ["zzz"]
    log_probs = []
    for i in range(1, len(tokens)):
        if n == 2:
            log_prob = model.score(tokens[i], tokens[i-1])
        elif n == 1:
            log_prob = model.score_unigram(tokens[i])
        else:
            raise ValueError("Only n=1 (unigram) and n=2 (bigram) are supported")

        log_probs.append(log_prob)


    return np.exp(-np.sum(log_probs) / len(log_probs))

example = "pengakuan tersebut diungkapkan dalam wawancara dengan media"

perplexity_bigram = calculate_perplexity(bm, example, n=2)
perplexity_unigram = calculate_perplexity(bm, example, n=1)

print(f"Perplexity Bigram: {perplexity_bigram}")
print(f"Perplexity Unigram: {perplexity_unigram}")

assert perplexity_bigram < perplexity_unigram, "Perplexity Bigram harus lebih kecil daripada Unigram"

Perplexity Bigram: 197.64950856658172
Perplexity Unigram: 4352.693526043406


### Soal 2.6 (3 poin)

Ambil 30 sampel dari data uji. Hitunglah nilai rata-rata berbobot (_weighted average_) perplexity per kalimat pada data uji dengan menggunakan jumlah kata per kalimat sebagai bobotnya.

Pastikan bahwa:
1. Metode tokenisasi dan preprocessing yang dilakukan sama dengan soal 2.2 dan 2.3.
1. Nilai perplexity dihitung untuk model unigram dan bigram dengan backoff.

_Petunjuk: Gunakan metode `.findall()`_

In [31]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict

def preprocess(text):
    return re.findall(r'\b\w\w+\b', text.lower())

class BigramModel:
    def __init__(self, unigram_df: pd.DataFrame, bigram_df: pd.DataFrame, alpha: float = 0.4):
        self.unigram_df = unigram_df
        self.bigram_df = bigram_df
        self.alpha = alpha

    def score_unigram(self, word: str) -> float:
        """Menghitung skor unigram dengan Laplace smoothing"""
        if word in self.unigram_df['word'].values:
            return np.log(self.unigram_df[self.unigram_df['word'] == word]['probability'].values[0])
        else:
            return np.log(1 / (self.unigram_df['freq'].sum() + len(self.unigram_df)))

    def score(self, word: str, prev_word: str) -> float:
        """Menghitung skor bigram dengan Stupid Backoff"""
        bigram_prob = self.get_bigram_prob(word, prev_word)
        if bigram_prob > 0:
            return np.log(bigram_prob)
        else:
            return np.log(self.alpha) + self.score_unigram(word)

    def get_bigram_prob(self, word: str, prev_word: str) -> float:
        """Mengambil probabilitas bigram"""
        bigram_count = self.bigram_df[(self.bigram_df['word'] == prev_word) & (self.bigram_df['next_word'] == word)]['freq']
        if len(bigram_count) > 0:
            bigram_freq = bigram_count.values[0]
            prev_word_freq = self.unigram_df[self.unigram_df['word'] == prev_word]['freq'].values[0]
            return bigram_freq / prev_word_freq
        else:
            return 0

    def sentence_score(self, sentence: str, n: int = 2) -> np.ndarray:
        tokens = ["ooo"] + preprocess(sentence) + ["zzz"]
        log_probs = []
        for i in range(1, len(tokens)):
            if n == 2:
                log_prob = self.score(tokens[i], tokens[i-1])
            elif n == 1:
                log_prob = self.score_unigram(tokens[i])
            else:
                raise ValueError("Only n=1 (unigram) and n=2 (bigram) are supported")


### Soal 2.7 (2 poin)

Berikan kesimpulan dari analisis yang sudah dilakukan pada bagian kedua dari tugas ini.

Pada tugas ini, penggunaan Regular Expressions sangat penting dalam tahap tokenisasi dan preprocessing teks. RegEx memungkinkan kita untuk mengidentifikasi dan memanipulasi pola dalam teks, seperti menghapus tanda baca atau memisahkan kata berdasarkan spasi. Ini membantu membersihkan dan mempersiapkan data sebelum digunakan dalam model bahasa n-gram. Meskipun RegEx sangat efektif dalam tugas-tugas pemrosesan teks, tantangan utamanya terletak pada keterbacaan dan kompleksitas sintaksisnya, yang dapat membingungkan bagi pengguna yang kurang berpengalaman.

Di sisi lain, model bahasa N-gram, seperti unigram dan bigram, digunakan untuk menghitung probabilitas kemunculan kata dalam sebuah kalimat berdasarkan konteks kata-kata sebelumnya. Model bigram lebih unggul dibandingkan unigram karena mampu menangkap ketergantungan antara kata yang berdekatan, menghasilkan prediksi yang lebih akurat. Meskipun demikian, model n-gram mengalami masalah sparsity, yang dapat diatasi dengan menggunakan teknik seperti smoothing. Hasil analisis menunjukkan bahwa model bigram dengan smoothing memberikan hasil yang lebih baik daripada model unigram dalam banyak kasus, meskipun masih terdapat keterbatasan dalam menangkap ketergantungan kata yang lebih jauh dalam teks.